In [1]:
import torch
from transformers import DistilBertModel, DistilBertTokenizer
from peft import LoraConfig, get_peft_model
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import librosa

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device : {device}")

Using device : cuda


In [3]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [4]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["attention.q_lin", "attention.v_lin"],
    lora_dropout=0.1
)
lora_model = get_peft_model(model, lora_config)
lora_model.to(device)
lora_model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): lora.Linear(
                (base_layer): Linear(in_features=768, out_features=768, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Lin

In [5]:
data = pd.read_csv("Dataset/Final Dysarthria-Non Dysarthria Speech Features.csv")

In [6]:
data.shape

(4282, 10002)

In [7]:
data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_9992,feature_9993,feature_9994,feature_9995,feature_9996,feature_9997,feature_9998,feature_9999,label,gender
0,-548.22070,-512.68445,-385.66476,-296.33347,-278.77200,-309.11040,-338.63345,-346.67023,-349.56323,-361.98727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Female
1,-576.23540,-570.52356,-563.91724,-467.63490,-427.55084,-401.95612,-375.35630,-434.04730,-455.38083,-450.59012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Female
2,-603.48150,-582.47845,-478.41200,-364.91333,-342.75296,-371.54932,-386.14365,-423.13620,-452.56390,-483.60983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Female
3,-616.47060,-587.69450,-589.40326,-587.86400,-579.49640,-549.48470,-470.16528,-390.60820,-333.97488,-301.32706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Female
4,-577.04650,-575.76090,-583.78546,-572.81540,-571.20715,-574.68500,-572.63000,-568.56250,-563.06085,-558.03760,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4277,-362.60742,-329.08860,-324.55450,-321.42572,-320.04068,-323.81036,-326.55673,-323.49510,-322.62424,-321.32010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Male
4278,-351.61120,-325.73544,-320.22736,-318.81357,-321.31387,-319.99472,-317.29916,-316.92715,-320.76240,-322.99478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Male
4279,-362.91858,-328.30580,-324.49197,-323.67795,-327.72690,-324.56934,-321.89062,-323.60220,-324.45087,-324.01114,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Male
4280,-346.29034,-320.42477,-319.24180,-317.26074,-320.89760,-320.12726,-317.15750,-310.82544,-315.87380,-322.21048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Male


In [8]:
X = data.drop(['gender','label'], axis=1)
y = data['label']

X_str = X.apply(lambda row:' '.join(row.astype(str)), axis=1)

In [9]:
texts = X_str.astype('str').to_list()
labels = y

In [10]:
%%time
batch_size = 8

all_embeddings = []
all_labels = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_labels = labels[i:i+batch_size]
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = lora_model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    all_embeddings.append(batch_embeddings)
    all_labels.extend(batch_labels)

all_embeddings = np.vstack(all_embeddings)
all_labels = np.array(all_labels)

X_train, X_test, y_train, y_test = train_test_split(all_embeddings, all_labels, test_size=0.2, random_state=42)

CPU times: total: 8min 31s
Wall time: 10min 30s


In [ ]:
%%time
rf_classifier = RandomForestClassifier(max_depth=20, min_samples_leaf=4, n_estimators=400)
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))
print("Random Forest Precision:", precision_score(y_test, rf_predictions))
print("Random Forest Recall:", recall_score(y_test, rf_predictions))
print("Random Forest F1-Score:", f1_score(y_test, rf_predictions))

In [ ]:
%%time
svm_classifier = SVC(C=1, kernel='linear', gamma=0.001, random_state=42)
svm_classifier.fit(X_train, y_train)
svm_predictions = svm_classifier.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))
print("SVM Precision:", precision_score(y_test, svm_predictions))
print("SVM Recall:", recall_score(y_test, svm_predictions))
print("SVM F1-Score:", f1_score(y_test, svm_predictions))

In [ ]:
%%time
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_test)
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_predictions))
print("Naive Bayes Precision:", precision_score(y_test, nb_predictions))
print("Naive Bayes Recall:", recall_score(y_test, nb_predictions))
print("Naive Bayes F1-Score:", f1_score(y_test, nb_predictions))

In [ ]:
%%time
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)
xgb_predictions = xgb_classifier.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_predictions))
print("XGBoost Precision:", precision_score(y_test, xgb_predictions))
print("XGBoost Recall:", recall_score(y_test, xgb_predictions))
print("XGBoost F1-Score:", f1_score(y_test, xgb_predictions))

Demo

In [ ]:
classes = {0 : "Non Dysarthria", 1 : "Dysarthria"}

In [ ]:
audio_path = "Sample Audio Files\\Dysarthria\\0001.wav"

In [ ]:
y, sr = librosa.load(audio_path, sr=None)
mfcc_features = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
chroma = librosa.feature.chroma_stft(y=y, sr=sr)
features = np.concatenate((mfcc_features, chroma), axis=0)
features_str = ' '.join(map(str, features.flatten()))

In [ ]:
inputs = tokenizer(features_str, padding=True, truncation=True, return_tensors="pt").to(device)
    
with torch.no_grad():
    outputs = lora_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

In [ ]:
xgb_prediction = xgb_classifier.predict(embeddings)
print(f"Prediction : {classes[xgb_prediction[0]]}")

In [ ]:
rf_prediction = rf_classifier.predict(embeddings)
print(f"Random Forest Prediction : {classes[rf_prediction[0]]}")

In [ ]:
svm_prediction = svm_classifier.predict(embeddings)
print(f"SVM Prediction : {classes[svm_prediction[0]]}")

In [ ]:
nb_prediction = nb_classifier.predict(embeddings)
print(f"Naive Bayes Prediction : {classes[nb_prediction[0]]}")